# **Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama**

## **İş Problemi**
#### Bir oyun şirketi müşterilerinin  bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona) oluşturmak ve bu yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.
#### **Örneğin**
#### Türkiye'den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

## **Veri Seti Hikayesi**
#### **Persona.csv** veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

## **Değişkenler**
#### **PRICE** - Müşterinin harcama tutarı
#### **SOURCE** - Müşterinin bağlandığı cihaz türü
#### **SEX** - Müşterinin cinsiyeti
#### **COUNTRY** - Müşterinin ülkesi
#### **AGE** - Müşterinin yaşı

### **Uygulama Öncesi Veri Seti:**

| PRICE   | SOURCE   | SEX   | COUNTRY   | AGE   |
|------------|------------|------------|------------|------------|
| 39 | android | male | bra | 17 |
| 39 | android | male | bra | 17 |
| 49 | android | male | bra | 17 |
| 29 | android | male | tur | 17 |
| 49 | android | male | tur | 17 |


### **Hedeflenen Çıktı:**

| costumers_level_based   | PRICE   | SEGMENT   |
|------------|------------|------------|
| BRA_ANDROID_FEMALE_0_18 | 35.6453 | B |
| BRA_ANDROID_FEMALE_19_23 | 34.0773 | C |
| BRA_ANDROID_FEMALE_24_30 | 33.8639 | C |
| BRA_ANDROID_FEMALE_31_40 | 34.8983 | B |
| BRA_ANDROID_FEMALE_41_66 | 36.7371 | A |


## **GÖREV-1:**  Aşağıdaki Soruları Yanıtlayınız

#### **1-** persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.read_csv("/kaggle/input/persona-csv/persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [2]:
df.shape

(5000, 5)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


#### **2-** Kaç unique SOURCE vardır? Frekansları nedir?

In [4]:
df["SOURCE"].nunique()

2

In [5]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

#### **3-** Kaç unique PRICE vardır?

In [6]:
df["PRICE"].nunique()

6

#### **4-** Hangi PRICE'dan kaçar tane satış gerçekleşmiştir?

In [7]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

#### **5-** Hangi ülkeden kaçar tane satış olmuştur?

In [8]:
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

#### **6-** Ülkelere göre satışlardan toplam ne kadar kazanılmıştır?

In [9]:
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


#### **7-** SOURCE türlerine göre satış sayıları nedir?

In [10]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [11]:
df.groupby("SOURCE").agg({"PRICE": "count"})

,PRICE
SOURCE,
android,2974
ios,2026


#### **8-** Ülkelere göre PRICE ortalamaları nedir?

In [12]:
df.groupby("COUNTRY").agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


#### **9-** SOURCE'lara göre PRICE ortalamaları nedir?

In [13]:
df.groupby(by=['SOURCE']).agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


#### **10-** COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

In [14]:
df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

## **GÖREV-2:**  COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?

### **Elde edilmesi gereken çıktı:**

|    |    |    |    |    | **PRICE**   |
|------------|------------|------------|------------|------------|------------|
| | **COUNTRY**| **SOURCE**| **SEX**| **AGE**| |
| 0 | bra | android | female | 15 | 38.71 |
| 1 |  |  |  | 16 | 35.94 |
| 2 |  |  |  | 17 | 35.66 |
| 3 |  |  |  | 18 | 32.25 |
| 4 |  |  |  | 19 | 35.20 |


In [15]:
df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

## **GÖREV-3:** Çıktıyı PRICE'a göre sıralayınız ve agg_df olarak kaydediniz.

### **Elde edilmesi gereken çıktı:**

|    |    |    |   |    | PRICE   |
|------------|------------|------------|------------|------------|------------|
| | **COUNTRY**| **SOURCE**| **SEX**| **AGE**| |
| 0 | bra | android | male | 46 | 59.0 |
| 1 | usa | android | male | 36 | 59.0 |
| 2 | fra | android | female | 24 | 59.0 |
| 3 | usa | ios | male | 32 | 54.0 |
| 4 | deu | android | female | 36 | 49.0 |


In [16]:
agg_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)

In [17]:
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


## **GÖREV-4:** Indekste yer alan isimleri değişken isimlerine çeviriniz.

In [18]:
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


## **GÖREV-5:** Age değişkenini kategorik değişkene çeviriniz ve **agg_df**'e ekleyiniz.

### **Elde edilmesi gereken örnek çıktı:**

| **COUNTRY**   | **SOURCE**   | **SEX**   | **AGE**   | **PRICE**   | **AGE_CAT**   |
|------------|------------|------------|------------|------------|------------|
| bra | android | male | 46 | 59.0 | 41_70 |
| usa | android | male | 36 | 59.0 | 31_40 |
| fra | android | female | 24 | 59.0 | 24_30 |
| usa | ios | male | 32 | 54.0 | 31_40 |
| deu | android | female | 36 | 49.0 | 31_40 |


In [19]:
bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

In [20]:
labels = ['0_18', '19_23', '24_30', '31_40', '41_'+ str(agg_df["AGE"].max())]

In [21]:
agg_df["age_cat"] = pd.cut(agg_df["AGE"], bins, labels=labels)

In [22]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [23]:
agg_df["age_cat"].value_counts()

age_cat
24_30    78
0_18     77
19_23    71
31_40    65
41_66    57
Name: count, dtype: int64

## **GÖREV-6:** Yeni seviye tabanlı müşterileri (persona) tanımlayınız ve customers_level_based değişkeni olarak veri setine ekleyiniz.

### **Elde edilmesi gereken çıktı:**

| **customers_level_based**   | **PRICE**   |
|------------|------------|
| BRA_ANDROID_MALE_41_66 | 59.0 |
| USA_ANDROID_MALE_31_40 | 59.0 |
| FRA_ANDROID_FEMALE_24_30 | 59.0 |
| USA_IOS_MALE_31_40 | 54.0 |
| DEU_ANDROID_FEMALE_31_40 | 49.0 |


In [24]:
agg_df["customers_level_based"] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'age_cat']].agg(lambda x: '_'.join(x).upper(), axis=1)

In [25]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


## **GÖREV-7:** Yeni müşterileri (personaları) segmentlere ayırınız.

* #### Yeni müşterileri (Örnek: USA_ANDROID_MALE_0_18) **PRICE**'a göre 4 segmente ayırınız.
* #### Segmentleri **SEGMENT** isimlendirmesi ile değişken olarak **agg_df**'e ekleyiniz.
* #### Segmentleri betimleyiniz. (Price mean, max, sum'larını alınız.)

In [26]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66,A
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40,A
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30,A
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40,A
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40,A


In [27]:
agg_df["SEGMENT"].value_counts()

SEGMENT
C    95
D    87
A    85
B    81
Name: count, dtype: int64

In [28]:
agg_df.groupby("SEGMENT", observed=False).agg({"PRICE": ["mean", "max", "sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        27.302596  31.105263  2375.325850
C        32.933339  34.000000  3128.667165
B        35.436170  37.000000  2870.329792
A        41.434736  59.000000  3521.952577

## **GÖREV-8:** Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.

* #### 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?
*  #### 35 yaşında IOS kullanan bir Fransız kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?

In [29]:
new_user_1 = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user_1]

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
18,tur,android,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,tur,android,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A


In [30]:
new_user_2 = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user_2]

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
208,fra,ios,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
221,fra,ios,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C
